In [1]:
import os, glob
import numpy as np
import torch
import numba
from numba import cuda

In [2]:
def boxes_iouBEV(boxes_a, boxes_b):
    """
    Args:
        boxes_a: (N, 7) [x, y, z, dx, dy, dz, heading]
        boxes_b: (M, 7) [x, y, z, dx, dy, dz, heading]

    Returns:
        ans_iou: (N, M)
    """
    assert boxes_a.shape[1] == boxes_b.shape[1] == 7
    boxes_a = torch.tensor(boxes_a.astype(np.float32)).cuda()
    boxes_b = torch.tensor(boxes_b.astype(np.float32)).cuda()
    
    # length overlap
    boxes_a_length_max = (boxes_a[:, 0] + boxes_a[:, 3] / 2).view(-1, 1)
    boxes_a_length_min = (boxes_a[:, 0] - boxes_a[:, 3] / 2).view(-1, 1)
    boxes_b_length_max = (boxes_b[:, 0] + boxes_b[:, 3] / 2).view(1, -1)
    boxes_b_length_min = (boxes_b[:, 0] - boxes_b[:, 3] / 2).view(1, -1)
    
    # widht overlap
    boxes_a_width_max = (boxes_a[:, 1] + boxes_a[:, 4] / 2).view(-1, 1)
    boxes_a_width_min = (boxes_a[:, 1] - boxes_a[:, 4] / 2).view(-1, 1)
    boxes_b_width_max = (boxes_b[:, 1] + boxes_b[:, 4] / 2).view(1, -1)
    boxes_b_width_min = (boxes_b[:, 1] - boxes_b[:, 4] / 2).view(1, -1)

    # bev overlap
    max_of_min_l = torch.max(boxes_a_length_min, boxes_b_length_min)
    min_of_max_l = torch.min(boxes_a_length_max, boxes_b_length_max)
    overlaps_l = torch.clamp(min_of_max_l - max_of_min_l, min=0)
    
    max_of_min_w = torch.max(boxes_a_width_min, boxes_b_width_min)
    min_of_max_w = torch.min(boxes_a_width_max, boxes_b_width_max)
    overlaps_w = torch.clamp(min_of_max_w - max_of_min_w, min=0)
    overlaps_bev =  overlaps_l * overlaps_w

    # BEV iou
    area_a = (boxes_a[:, 3] * boxes_a[:, 4]).view(-1, 1)
    area_b = (boxes_b[:, 3] * boxes_b[:, 4]).view(1, -1)

    iou2d = overlaps_bev / torch.clamp(area_a + area_b - overlaps_bev, min=1e-6)

    return iou2d.cpu().numpy()

In [3]:
def boxes_iou3d_gpu(boxes_a, boxes_b):
    """
    Args:
        boxes_a: (N, 7) [x, y, z, dx, dy, dz, heading]
        boxes_b: (M, 7) [x, y, z, dx, dy, dz, heading]

    Returns:
        ans_iou: (N, M)
    """
    assert boxes_a.shape[1] == boxes_b.shape[1] == 7
    boxes_a = torch.tensor(boxes_a.astype(np.float32)).cuda()
    boxes_b = torch.tensor(boxes_b.astype(np.float32)).cuda()
    
    # length overlap
    boxes_a_length_max = (boxes_a[:, 0] + boxes_a[:, 3] / 2).view(-1, 1)
    boxes_a_length_min = (boxes_a[:, 0] - boxes_a[:, 3] / 2).view(-1, 1)
    boxes_b_length_max = (boxes_b[:, 0] + boxes_b[:, 3] / 2).view(1, -1)
    boxes_b_length_min = (boxes_b[:, 0] - boxes_b[:, 3] / 2).view(1, -1)
    
    # widht overlap
    boxes_a_width_max = (boxes_a[:, 1] + boxes_a[:, 4] / 2).view(-1, 1)
    boxes_a_width_min = (boxes_a[:, 1] - boxes_a[:, 4] / 2).view(-1, 1)
    boxes_b_width_max = (boxes_b[:, 1] + boxes_b[:, 4] / 2).view(1, -1)
    boxes_b_width_min = (boxes_b[:, 1] - boxes_b[:, 4] / 2).view(1, -1)

    # height overlap
    boxes_a_height_max = (boxes_a[:, 2] + boxes_a[:, 5] / 2).view(-1, 1)
    boxes_a_height_min = (boxes_a[:, 2] - boxes_a[:, 5] / 2).view(-1, 1)
    boxes_b_height_max = (boxes_b[:, 2] + boxes_b[:, 5] / 2).view(1, -1)
    boxes_b_height_min = (boxes_b[:, 2] - boxes_b[:, 5] / 2).view(1, -1)

    # bev overlap
    max_of_min_l = torch.max(boxes_a_length_min, boxes_b_length_min)
    min_of_max_l = torch.min(boxes_a_length_max, boxes_b_length_max)
    overlaps_l = torch.clamp(min_of_max_l - max_of_min_l, min=0)
    
    max_of_min_w = torch.max(boxes_a_width_min, boxes_b_width_min)
    min_of_max_w = torch.min(boxes_a_width_max, boxes_b_width_max)
    overlaps_w = torch.clamp(min_of_max_w - max_of_min_w, min=0)
    overlaps_bev =  overlaps_l * overlaps_w

    max_of_min = torch.max(boxes_a_height_min, boxes_b_height_min)
    min_of_max = torch.min(boxes_a_height_max, boxes_b_height_max)
    overlaps_h = torch.clamp(min_of_max - max_of_min, min=0)

    # 3d iou
    overlaps_3d = overlaps_bev * overlaps_h
    vol_a = (boxes_a[:, 3] * boxes_a[:, 4] * boxes_a[:, 5]).view(-1, 1)
    vol_b = (boxes_b[:, 3] * boxes_b[:, 4] * boxes_b[:, 5]).view(1, -1)

    iou3d = overlaps_3d / torch.clamp(vol_a + vol_b - overlaps_3d, min=1e-6)

    return iou3d.cpu().numpy()

# Folder

In [115]:
label_files = glob.glob('/nas2/YJ2/Benchmark_all/5.3D_Bounding_Box/labels/*.txt')
for label_file in label_files :
    with open(label_file, 'r') as f :
        lines = f.readlines()
        boxes=[]
        for line in lines :
            label = line.strip().split(' ')
            h, w, l = float(label[8]), float(label[10]), float(label[9])
            x, y, z = float(label[11]), float(label[12]), float(label[13])

            box = np.array((x, y, z, l, w, h, 0.0), dtype=np.float32)
            boxes.append(box)
        boxes = np.array(boxes)
        iou_3d = boxes_iou3d_gpu(boxes, boxes)
        iou_2d = boxes_iouBEV(boxes, boxes)
        
        np.fill_diagonal(iou_3d, 0)
        np.fill_diagonal(iou_2d, 0)

        total_tree = iou_3d.shape[0]
        non_zero_cnt = np.count_nonzero(iou_3d, axis=0)
        num_overlap_tree = np.where(non_zero_cnt > 0)[0].shape[0]

        if num_overlap_tree != 0 :
            avg_iou3d = iou_3d.sum() / non_zero_cnt.sum()
            avg_iou2d = iou_2d.sum() / non_zero_cnt.sum()
            avg_num_overlaps = non_zero_cnt.sum() / num_overlap_tree
        else :
            avg_iou3d = 0.0
            avg_iou2d = 0.0
            avg_num_overlaps = 0.0
            
        print("{}, num_total:{}, num_overlap_trees:{}, avg.overlaps:{:.2f}, avg/max_IoU2d: {:.2f}/{:.2f}, avg/max_IoU3d: {:.2f}/{:.2f}".format(
            os.path.split(label_file)[1],
            total_tree,
            num_overlap_tree,
            avg_num_overlaps,
            avg_iou2d,
            iou_2d.max(),
            avg_iou3d,
            iou_3d.max()
            ))
       

58.txt, num_total:5, num_overlap_trees:4, avg.overlaps:3.00, avg/max_IoU2d: 0.13/0.30, avg/max_IoU3d: 0.12/0.24
59.txt, num_total:2, num_overlap_trees:0, avg.overlaps:0.00, avg/max_IoU2d: 0.00/0.00, avg/max_IoU3d: 0.00/0.00
98.txt, num_total:5, num_overlap_trees:4, avg.overlaps:1.00, avg/max_IoU2d: 0.01/0.01, avg/max_IoU3d: 0.01/0.01
99.txt, num_total:8, num_overlap_trees:6, avg.overlaps:2.33, avg/max_IoU2d: 0.12/0.24, avg/max_IoU3d: 0.12/0.24
100.txt, num_total:7, num_overlap_trees:2, avg.overlaps:1.00, avg/max_IoU2d: 0.73/0.73, avg/max_IoU3d: 0.71/0.71
101.txt, num_total:9, num_overlap_trees:9, avg.overlaps:1.56, avg/max_IoU2d: 0.19/0.30, avg/max_IoU3d: 0.17/0.28
136.txt, num_total:2, num_overlap_trees:2, avg.overlaps:1.00, avg/max_IoU2d: 0.01/0.01, avg/max_IoU3d: 0.01/0.01
137.txt, num_total:1, num_overlap_trees:0, avg.overlaps:0.00, avg/max_IoU2d: 0.00/0.00, avg/max_IoU3d: 0.00/0.00
138.txt, num_total:5, num_overlap_trees:4, avg.overlaps:1.00, avg/max_IoU2d: 0.07/0.10, avg/max_IoU3

# File

In [117]:
num = 774
ROOT= '/nas2/YJ2/Benchmark_all/5.3D_Bounding_Box/labels'
file_nane = '{}.txt'.format(num)
label_file = os.path.join(ROOT, file_nane)
with open(label_file, 'r') as f :
    lines = f.readlines()
    boxes=[]
    for line in lines :
        label = line.strip().split(' ')
        h, w, l = float(label[8]), float(label[10]), float(label[9])
        x, y, z = float(label[11]), float(label[12]), float(label[13])

        box = np.array((x, y, z, l, w, h, 0.0), dtype=np.float32)
        boxes.append(box)
    boxes = np.array(boxes)
    iou_3d = boxes_iou3d_gpu(boxes, boxes)
    iou_2d = boxes_iouBEV(boxes, boxes)

    np.fill_diagonal(iou_3d, 0)
    np.fill_diagonal(iou_2d, 0)

    total_tree = iou_3d.shape[0]
    non_zero_cnt = np.count_nonzero(iou_3d, axis=0)
    num_overlap_tree = np.where(non_zero_cnt > 0)[0].shape[0]

    if num_overlap_tree != 0 :
        avg_iou3d = iou_3d.sum() / non_zero_cnt.sum()
        avg_iou2d = iou_2d.sum() / non_zero_cnt.sum()
        avg_num_overlaps = non_zero_cnt.sum() / num_overlap_tree
    else :
        avg_iou3d = 0.0
        avg_iou2d = 0.0
        avg_num_overlaps = 0.0

    print("{}, num_total:{}, num_overlap_trees:{}, avg.overlaps:{:.2f}, avg/max_IoU2d: {:.2f}/{:.2f}, avg/max_IoU3d: {:.2f}/{:.2f}".format(
        os.path.split(label_file)[1],
        total_tree,
        num_overlap_tree,
        avg_num_overlaps,
        avg_iou2d,
        iou_2d.max(),
        avg_iou3d,
        iou_3d.max()
        ))

774.txt, num_total:10, num_overlap_trees:10, avg.overlaps:1.80, avg/max_IoU2d: 0.82/1.00, avg/max_IoU3d: 0.75/1.00


In [126]:
# IoU 2D matrix
row, col = iou_2d.shape
for r in range(row) :
    for c in range(col) :
        print ('{:.2f}, '.format(iou_2d[r][c]), end='')
    print()

0.00, 0.00, 0.00, 0.00, 0.84, 0.00, 0.00, 0.00, 0.00, 0.00, 
0.00, 0.00, 0.00, 0.00, 0.00, 0.94, 0.68, 0.68, 0.00, 0.00, 
0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 
0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 
0.84, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 
0.00, 0.94, 0.00, 0.00, 0.00, 0.00, 0.63, 0.63, 0.00, 0.00, 
0.00, 0.68, 0.00, 0.00, 0.00, 0.63, 0.00, 1.00, 0.00, 0.00, 
0.00, 0.68, 0.00, 0.00, 0.00, 0.63, 1.00, 0.00, 0.00, 0.00, 
0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 
0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 


In [121]:
lines

['tree 0 0 1.57 58.6581 44.80620000046 60 41.96850000042 5.504000854 1.3419 2.83770000003 59.34671199997 43.39965999965 316.697999573 0\n',
 'tree 0 0 1.57 56.55700000003 52.24129999988 59.42660000001 49.05250000022 5.2 2.86959999998 3.18879999965 57.99172499997 50.64668000024 317.2 0\n',
 'tree 0 0 1.57 53.60970000003 60 57.13780000003 56.30950000044 5.784003296 3.5281 3.69049999956 55.36749900004 58.17623000033 316.897998352 0\n',
 'tree 0 0 1.57 53.60970000003 60 57.13780000003 56.30950000044 5.784003296 3.5281 3.69049999956 55.36749900004 58.17623000033 316.897998352 0\n',
 'tree 0 0 1.57 58.52410000004 44.80620000046 60 41.74990000017 5.583995361 1.47589999996 3.05630000029 59.28125500004 43.29126999993 316.7280023195 0\n',
 'tree 0 0 1.57 56.37349999999 52.2444000002 59.42660000001 49.04829999991 4.824000854 3.05310000002 3.19610000029 57.900395 50.64615000039 316.357999573 0\n',
 'tree 0 0 1.57 57.05660000001 52.21769999992 59.40020000003 49.57820000034 4.803996582 2.34360000002